# METIS Intro to Data Science Project

## By: Ben McGauhey

### Seattle, WA Fall-2017

### Import our libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import folium
import matplotlib.pyplot as plt
%matplotlib inline 

### Read our csv files into a dataframe

In [2]:
rawCallData = pd.read_csv('Data/Seattle_Police_Department_911_Incident_Response.csv')

/Users/BenM/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### An error right out of the gate, let's see what the problem is

In [3]:
rawCallData.dtypes

CAD CDW ID                      object
CAD Event Number                 int64
General Offense Number           int64
Event Clearance Code           float64
Event Clearance Description     object
Event Clearance SubGroup        object
Event Clearance Group           object
Event Clearance Date            object
Hundred Block Location          object
District/Sector                 object
Zone/Beat                       object
Census Tract                   float64
Longitude                      float64
Latitude                       float64
Incident Location               object
Initial Type Description        object
Initial Type Subgroup           object
Initial Type Group              object
At Scene Time                   object
dtype: object

### Let's start by cleaning up those column names 

In [4]:
rawCallData.columns = rawCallData.columns.str.lower().str.replace(" ", "_").str.replace("/", "_") 
# Remove the white space and slashes in our column names
rawCallData.columns # Check our work

Index(['cad_cdw_id', 'cad_event_number', 'general_offense_number',
       'event_clearance_code', 'event_clearance_description',
       'event_clearance_subgroup', 'event_clearance_group',
       'event_clearance_date', 'hundred_block_location', 'district_sector',
       'zone_beat', 'census_tract', 'longitude', 'latitude',
       'incident_location', 'initial_type_description',
       'initial_type_subgroup', 'initial_type_group', 'at_scene_time'],
      dtype='object')

### That's better,  but an attempt to coerce the datatype leads us to the problems,
Let's take a minute and fix these

In [5]:
rawCallData.loc[rawCallData.cad_cdw_id == 'ï»¿1875911'] # Look at the row from the error message

,cad_cdw_id,cad_event_number,general_offense_number,event_clearance_code,event_clearance_description,event_clearance_subgroup,event_clearance_group,event_clearance_date,hundred_block_location,district_sector,zone_beat,census_tract,longitude,latitude,incident_location,initial_type_description,initial_type_subgroup,initial_type_group,at_scene_time
857606,ï»¿1875911,14000012658,201412658,41.0,"HARASSMENT, THREATS","THREATS, HARASSMENT","THREATS, HARASSMENT",01/13/2014 12:51:00 AM,39XX BLOCK OF S JUNEAU ST,S,S1,10300.3006,-122.282103,47.549712,"(47.549711879, -122.282103194)",THREATS (INCLS IN-PERSON/BY PHONE/IN WRITING),"THREATS, HARASSMENT","THREATS, HARASSMENT",01/12/2014 08:00:00 PM


In [6]:
rawCallData.loc[857606,'cad_cdw_id'] = '1875911' # Change the value

In [7]:
rawCallData.loc[rawCallData.cad_cdw_id == '\ufeff15736'] # Look at the next error

,cad_cdw_id,cad_event_number,general_offense_number,event_clearance_code,event_clearance_description,event_clearance_subgroup,event_clearance_group,event_clearance_date,hundred_block_location,district_sector,zone_beat,census_tract,longitude,latitude,incident_location,initial_type_description,initial_type_subgroup,initial_type_group,at_scene_time
0,﻿15736,10000246357,2010246357,242.0,FIGHT DISTURBANCE,DISTURBANCES,DISTURBANCES,07/17/2010 08:49:00 PM,3XX BLOCK OF PINE ST,M,M2,8100.2001,-122.338147,47.610975,"(47.610975163, -122.338146748)",NaN,NaN,NaN,NaN


In [8]:
rawCallData.loc[0, 'cad_cdw_id'] = '15736' # Fix it

In [9]:
rawCallData.loc[rawCallData.cad_cdw_id == 'ï»¿880295'] # Whew, one more time

,cad_cdw_id,cad_event_number,general_offense_number,event_clearance_code,event_clearance_description,event_clearance_subgroup,event_clearance_group,event_clearance_date,hundred_block_location,district_sector,zone_beat,census_tract,longitude,latitude,incident_location,initial_type_description,initial_type_subgroup,initial_type_group,at_scene_time
926010,ï»¿880295,12000276191,2012276191,71.0,AUTO THEFT,AUTO THEFTS,AUTO THEFTS,08/19/2012 03:16:00 PM,18XX BLOCK OF EASTLAKE AVE E,D,D2,6600.2,-122.325401,47.63537,"(47.635369535, -122.325400816)",AUTO THEFT - VEH THEFT OR THEFT & RECOVERY,AUTO THEFTS,AUTO RECOVERIES,08/19/2012 02:38:00 PM


In [10]:
rawCallData.loc[926010, 'cad_cdw_id'] = '880295' # Last one to fix

In [11]:
rawCallData.cad_cdw_id = rawCallData.cad_cdw_id.astype('int64', errors='raise') # Coerce the data to int64

### Fixed that, on we go to our next dataframe

In [12]:
rawWeatherData = pd.read_csv('Data/Seattle_Weather.csv')

### Lets take a look at our data frames to see what we've got 

In [13]:
rawCallData.head()

,cad_cdw_id,cad_event_number,general_offense_number,event_clearance_code,event_clearance_description,event_clearance_subgroup,event_clearance_group,event_clearance_date,hundred_block_location,district_sector,zone_beat,census_tract,longitude,latitude,incident_location,initial_type_description,initial_type_subgroup,initial_type_group,at_scene_time
0,15736,10000246357,2010246357,242.0,FIGHT DISTURBANCE,DISTURBANCES,DISTURBANCES,07/17/2010 08:49:00 PM,3XX BLOCK OF PINE ST,M,M2,8100.2001,-122.338147,47.610975,"(47.610975163, -122.338146748)",NaN,NaN,NaN,NaN
1,15737,10000246471,2010246471,65.0,THEFT - MISCELLANEOUS,THEFT,OTHER PROPERTY,07/17/2010 08:50:00 PM,36XX BLOCK OF DISCOVERY PARK BLVD,Q,Q1,5700.1012,-122.404613,47.658325,"(47.658324899, -122.404612874)",NaN,NaN,NaN,NaN
2,15738,10000246255,2010246255,250.0,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",07/17/2010 08:55:00 PM,21XX BLOCK OF 3RD AVE,M,M2,7200.2025,-122.342843,47.613551,"(47.613551471, -122.342843234)",NaN,NaN,NaN,NaN
3,15739,10000246473,2010246473,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,07/17/2010 09:00:00 PM,7XX BLOCK OF ROY ST,D,D1,7200.1002,-122.341847,47.625401,"(47.625401388, -122.341846999)",NaN,NaN,NaN,NaN
4,15740,10000246330,2010246330,250.0,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",07/17/2010 09:00:00 PM,9XX BLOCK OF ALOHA ST,D,D1,6700.1009,-122.339709,47.627425,"(47.627424837, -122.339708605)",NaN,NaN,NaN,NaN


In [14]:
rawWeatherData.head()

,dt,dt_iso,city_id,city_name,lat,lon,temp,temp_min,temp_max,pressure,...,rain_today,snow_1h,snow_3h,snow_24h,snow_today,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1349096400,2012-10-01 13:00:00 +0000 UTC,5809844,NaN,NaN,NaN,281.80,278.15,287.59,1027,...,NaN,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
1,1349186400,2012-10-02 14:00:00 +0000 UTC,5809844,NaN,NaN,NaN,281.62,278.15,286.48,1046,...,NaN,NaN,NaN,NaN,NaN,66,800,Clear,sky is clear,02d
2,1349190000,2012-10-02 15:00:00 +0000 UTC,5809844,NaN,NaN,NaN,282.71,279.82,289.82,1026,...,NaN,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01d
3,1349193600,2012-10-02 16:00:00 +0000 UTC,5809844,NaN,NaN,NaN,285.05,281.48,293.15,1026,...,NaN,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01d
4,1349197200,2012-10-02 17:00:00 +0000 UTC,5809844,NaN,NaN,NaN,287.97,282.59,296.48,1027,...,NaN,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01d


### Lets examine the NaN values, starting with rawCallData

In [15]:
print(rawCallData.isnull().sum()) # Get column wise sums of the NaN's

cad_cdw_id                           0
cad_event_number                     0
general_offense_number               0
event_clearance_code             10797
event_clearance_description      10798
event_clearance_subgroup         10798
event_clearance_group            10798
event_clearance_date             10951
hundred_block_location            3487
district_sector                   1162
zone_beat                            1
census_tract                      2792
longitude                            1
latitude                             1
incident_location                    1
initial_type_description        577813
initial_type_subgroup           577813
initial_type_group              577813
at_scene_time                  1029344
dtype: int64


### We have a significant number of records missing some information
They seem to be clustered. There is a cluster of 10,790+ records missing event data, 577,813 records missing initial data and a third large cluster where most of the records are missing  at scene time. The event clearance and initial type columns seem to all describe the same data, what the call was about. The at scene time and call clearance date similiarly overlap. The bad news is, most of the missing data pertains to what we care about, which is what happened and when. They good news is these columns seem to provide redundant information, so we can use one to impute the other. We can see a few columns provide no relevant information and can be dropped, subgroup and group. Let's see if there are any records that provide no relevant information.

In [16]:
rawCallData.drop(['event_clearance_subgroup', 'event_clearance_group', 'initial_type_subgroup', 
                  'initial_type_group'], axis=1,inplace=True) # Drop our un-necessary columns

In [17]:
mask = (rawCallData.event_clearance_description.isnull()) & (rawCallData.initial_type_description.isnull())
noEvent = rawCallData[mask] # Our mask selects records that have a null value in both description columns
print(noEvent.shape) # Check the size of our haul

(932, 15)


### We have 932 records with no event descriptor, we will have to remove them,
While not shown here we also checked against the other descriptive columns

In [18]:
print(rawCallData.shape) # Check original datafile shape
rawCallData = rawCallData[~mask] # Remove by selecting the inverse of our mask as subset
print(rawCallData.shape) # Verify our subtraction

(1445066, 15)
(1444134, 15)


In [19]:
mask2 = (rawCallData.at_scene_time.isnull()) & (rawCallData.event_clearance_date.isnull())
noTime = rawCallData[mask2] # Our mask selects records that have a null value in both time columns
print(noTime.shape) # Check the size of our haul

(9, 15)


### We have 9 records with no time, we will have to remove them

In [20]:
print(rawCallData.shape) # Check original datafile shape
rawCallData = rawCallData[~mask2] # Remove by selecting the inverse of our mask as subset
print(rawCallData.shape) # Verify our subtraction

(1444134, 15)
(1444125, 15)


### Looking again at the time values,
We can only use the overlap in our data sets date range, so lets switch gears and make sure that our dataframes cover the same time period. Continuing to clean up the null values in rawCallData could be a waste if those records don't overlap with our weather data. We will start by cleaning up the assorted time columns and getting one formatted datetime column.

In [21]:
rawCallData['formatted_time'] = pd.to_datetime(rawCallData.event_clearance_date, 
                                               format='%m/%d/%Y %I:%M:%S %p', errors='coerce') # This will convert the 
# Time column by specifying the format as 01/01/2017 01:02:03 AM, which will speed it up exponentially

In [22]:
print(rawCallData.formatted_time.head()) # Check our work
print("Total errors in conversion : " + str(rawCallData.formatted_time.isnull().sum()))

0   2010-07-17 20:49:00
1   2010-07-17 20:50:00
2   2010-07-17 20:55:00
3   2010-07-17 21:00:00
4   2010-07-17 21:00:00
Name: formatted_time, dtype: datetime64[ns]
Total errors in conversion : 10010


In [23]:
rawWeatherData['formatted_time'] = pd.to_datetime(rawWeatherData.dt, unit='s') # Convert using the unix timecode column

In [24]:
print(rawWeatherData.formatted_time.head()) # Check our work
print(rawWeatherData.head())

0   2012-10-01 13:00:00
1   2012-10-02 14:00:00
2   2012-10-02 15:00:00
3   2012-10-02 16:00:00
4   2012-10-02 17:00:00
Name: formatted_time, dtype: datetime64[ns]
           dt                         dt_iso  city_id  city_name  lat  lon  \
0  1349096400  2012-10-01 13:00:00 +0000 UTC  5809844        NaN  NaN  NaN   
1  1349186400  2012-10-02 14:00:00 +0000 UTC  5809844        NaN  NaN  NaN   
2  1349190000  2012-10-02 15:00:00 +0000 UTC  5809844        NaN  NaN  NaN   
3  1349193600  2012-10-02 16:00:00 +0000 UTC  5809844        NaN  NaN  NaN   
4  1349197200  2012-10-02 17:00:00 +0000 UTC  5809844        NaN  NaN  NaN   

     temp  temp_min  temp_max  pressure         ...          snow_1h  snow_3h  \
0  281.80    278.15    287.59      1027         ...              NaN      NaN   
1  281.62    278.15    286.48      1046         ...              NaN      NaN   
2  282.71    279.82    289.82      1026         ...              NaN      NaN   
3  285.05    281.48    293.15      1026    

### We can see this matches the UTC time given in the next column, so we need to convert to  local.
This is a three step process, first changing our naive time to UTC zone aware time, then converting to PDT through a lambda function. We can't use tz_convert inline because it defaults to changing the index rather than the value. We then undo making it zone aware with tz_localize as another lambda function.

In [25]:
rawWeatherData.formatted_time = rawWeatherData.formatted_time.dt.tz_localize('UTC') # Set to UTC zone aware
rawWeatherData.formatted_time = rawWeatherData.formatted_time.apply(lambda x: x.tz_convert('America/Los_Angeles'))
rawWeatherData.formatted_time = rawWeatherData.formatted_time.apply(lambda x: x.tz_localize(None))
# Then convert to our local timezone, and finally remove the zone awareness so everything is naive datetime 64

rawWeatherData.formatted_time.head() # Check our work

0   2012-10-01 06:00:00
1   2012-10-02 07:00:00
2   2012-10-02 08:00:00
3   2012-10-02 09:00:00
4   2012-10-02 10:00:00
Name: formatted_time, dtype: datetime64[ns]

### That looks much better, now we can apply a mask to subset our call data

In [26]:
mask3 = (rawCallData.formatted_time > '2012-10-01 13:00:00') # We only want from the start of our weather data
rawCallData = rawCallData[mask3]

In [27]:
rawCallData.shape # Check and see how many records we are left with

(879393, 16)

In [28]:
rawCallData.isnull().sum() # Where are we now with our null values?

cad_cdw_id                          0
cad_event_number                    0
general_offense_number              0
event_clearance_code                0
event_clearance_description         0
event_clearance_date                0
hundred_block_location              1
district_sector                   937
zone_beat                           0
census_tract                     1615
longitude                           0
latitude                            0
incident_location                   0
initial_type_description        72663
at_scene_time                  516351
formatted_time                      0
dtype: int64

# This is an important learning note:
# _Always start with a plan_
To be honest when cleaning this data, I just started with the 911 call data, looking for ways to clean out the NaN values. I'm leaving the original plan in this notebook, which was to use one column to impute the other, without regard to the bigger picture. This involved some neat data cleaning tricks, but in the end would have been completely wasted work. Without the corresponding weather data, those records are worthless to us. Luckily, I stopped and thought it out after hitting a few road blocks, realizing that the work might not be necessary. Turns out had I stopped and made a plan in the first place, I could've save quite a bit of time.
## _Always make a plan and then tackle your data, don't just start coding_
### Duly noted, on we go

### The columns with NaN values still are offering us redundant information about the description or location and can be dropped

In [29]:
rawCallData.drop(['event_clearance_date', 'hundred_block_location', 'district_sector', 'census_tract',
                 'incident_location', 'initial_type_description', 'at_scene_time'], inplace=True, axis=1) # Drop em
rawCallData.shape # Trust but verify

(879393, 9)

In [30]:
print(rawCallData.isnull().sum()) # Check our null value counts again
rawCallData.head()

cad_cdw_id                     0
cad_event_number               0
general_offense_number         0
event_clearance_code           0
event_clearance_description    0
zone_beat                      0
longitude                      0
latitude                       0
formatted_time                 0
dtype: int64


,cad_cdw_id,cad_event_number,general_offense_number,event_clearance_code,event_clearance_description,zone_beat,longitude,latitude,formatted_time
49,1658027,16000028163,201628163,245.0,"DISTURBANCE, OTHER",N2,-122.347770,47.731678,2016-01-24 11:54:55
70,1658028,16000028161,201628161,280.0,SUSPICIOUS PERSON,S1,-122.280685,47.523026,2016-01-24 11:57:35
105,1658029,16000028159,201628159,65.0,THEFT - MISCELLANEOUS,M1,-122.342000,47.609535,2016-01-24 11:54:28
190,1658030,16000028134,201628134,200.0,ALACAD - COMMERCIAL BURGLARY (FALSE),U1,-122.313020,47.668995,2016-01-24 11:53:22
255,1658031,16000028114,201628114,161.0,TRESPASS,D2,-122.344670,47.641580,2016-01-24 11:59:45


### Now we can look to see if the event numbers and ID numbers offer unique ID's

In [31]:
print(rawCallData.cad_cdw_id.nunique()) # Lets look at how many unique records we have to search for duplicates
print(rawCallData.cad_event_number.nunique())
print(rawCallData.general_offense_number.nunique())

879366
878480
878480


In [32]:
rawCallData[rawCallData.cad_cdw_id.duplicated()] # Are they duplicate records or just cdw_id?

,cad_cdw_id,cad_event_number,general_offense_number,event_clearance_code,event_clearance_description,zone_beat,longitude,latitude,formatted_time
835500,1873964,14000010381,201410381,64.0,SHOPLIFT,N3,-122.324615,47.708603,2014-01-10 15:44:00
835504,1873952,16000405795,2016405795,280.0,SUSPICIOUS PERSON,B1,-122.385630,47.670610,2016-11-09 11:31:57
835506,1873969,16000405783,2016405783,470.0,PARKING VIOLATION (EXCEPT ABANDONED VEHICLES),C3,-122.295530,47.623010,2016-11-09 11:44:46
1131303,1873957,16000405748,2016405748,245.0,"DISTURBANCE, OTHER",B2,-122.373180,47.673977,2016-11-09 11:32:22
1131304,1873958,16000405734,2016405734,470.0,PARKING VIOLATION (EXCEPT ABANDONED VEHICLES),B1,-122.378334,47.668423,2016-11-09 11:22:51
1133136,1873928,16000405451,2016405451,450.0,DRIVING WHILE UNDER INFLUENCE (DUI),U1,-122.313030,47.669440,2016-11-09 09:47:05
1133166,1873926,16000405578,2016405578,64.0,SHOPLIFT,D1,-122.348540,47.616524,2016-11-09 09:48:19
1133167,1873927,16000405572,2016405572,282.0,SUSPICIOUS CIRCUMSTANCES - BUILDING (OPEN DOOR...,D1,-122.346100,47.615475,2016-11-09 09:50:21
1133185,1873923,16000405632,2016405632,430.0,MOTOR VEHICLE COLLISION,F1,-122.332140,47.534145,2016-11-09 09:59:05
1133280,1873921,16000405658,2016405658,63.0,THEFT - CAR PROWL,U1,-122.302925,47.669205,2016-11-09 09:51:28


In [33]:
rawCallData.sort_values(by=['cad_cdw_id'],inplace=True) # Sort by the cad_cdw_id

In [34]:
 rawCallData.loc[rawCallData.cad_cdw_id.duplicated(keep=False) == True] # Now we can see our duplicates

,cad_cdw_id,cad_event_number,general_offense_number,event_clearance_code,event_clearance_description,zone_beat,longitude,latitude,formatted_time
835468,1873921,14000010389,201410389,184.0,"NARCOTICS, OTHER",K1,-122.338247,47.610246,2014-01-10 15:03:00
1133280,1873921,16000405658,2016405658,63.0,THEFT - CAR PROWL,U1,-122.302925,47.669205,2016-11-09 09:51:28
1133281,1873922,16000405655,2016405655,460.0,TRAFFIC (MOVING) VIOLATION,J1,-122.364720,47.700382,2016-11-09 10:01:35
835469,1873922,14000010384,201410384,64.0,SHOPLIFT,F2,-122.369349,47.521037,2014-01-10 14:51:00
1133185,1873923,16000405632,2016405632,430.0,MOTOR VEHICLE COLLISION,F1,-122.332140,47.534145,2016-11-09 09:59:05
835470,1873923,14000010370,201410370,470.0,PARKING VIOLATION (EXCEPT ABANDONED VEHICLES),D1,-122.344270,47.626270,2014-01-10 14:57:00
1133166,1873926,16000405578,2016405578,64.0,SHOPLIFT,D1,-122.348540,47.616524,2016-11-09 09:48:19
835471,1873926,14000010083,201410083,363.0,MISSING PERSON,K2,-122.331124,47.601718,2014-01-10 14:58:00
835472,1873927,14000010066,201410066,363.0,MISSING PERSON,G3,-122.297222,47.593315,2014-01-10 14:55:00
1133167,1873927,16000405572,2016405572,282.0,SUSPICIOUS CIRCUMSTANCES - BUILDING (OPEN DOOR...,D1,-122.346100,47.615475,2016-11-09 09:50:21


### Well we can see here that these are definitely different events, so our ID numbers are not unique identifiers, lets check cad_event number next

In [35]:
rawCallData.cad_event_number = rawCallData.cad_event_number.astype('int64', errors='raise') # Coerce to int64 first

In [36]:
rawCallData.sort_values(by=['cad_event_number'], inplace=True) # Sort to compare

In [37]:
rawCallData.loc[rawCallData.cad_event_number.duplicated(keep=False) == True] # List them to review

,cad_cdw_id,cad_event_number,general_offense_number,event_clearance_code,event_clearance_description,zone_beat,longitude,latitude,formatted_time
997757,931933,12000252943,2012252943,63.0,THEFT - CAR PROWL,G1,-122.302771,47.608007,2012-10-15 11:05:00
1285548,579794,12000252943,2012252943,63.0,THEFT - CAR PROWL,G2,-122.302771,47.608007,2012-10-15 11:05:00
1185456,2142696,14000275708,2014275708,63.0,THEFT - CAR PROWL,N1,-122.338090,47.712302,2014-08-19 13:25:00
1185694,2143025,14000275708,2014275708,65.0,THEFT - MISCELLANEOUS,N1,-122.338090,47.712302,2014-08-19 19:35:00
1199203,2157228,14000314989,2014314989,71.0,AUTO THEFT,O3,-122.321684,47.546745,2014-09-20 13:08:00
1200366,2158430,14000314989,2014314989,71.0,AUTO THEFT,O3,-122.321684,47.546745,2014-09-20 13:08:00
1264603,2230530,14000422775,2014422775,71.0,AUTO THEFT,O3,-122.293751,47.538912,2014-12-22 22:02:00
1266669,2234938,14000422775,2014422775,71.0,AUTO THEFT,O3,-122.293751,47.538912,2014-12-22 22:02:00
1265028,2230986,14000423288,2014423288,71.0,AUTO THEFT,N3,-122.324615,47.708603,2014-12-23 15:02:00
1266801,2235076,14000423288,2014423288,71.0,AUTO THEFT,N3,-122.324615,47.708603,2014-12-23 15:02:00


### So here we have duplicated records, and a valid ID column, 
Which we could just use the index and get rid of these ID columns altogether, but I would like to have a name for each record that will travel with the record as it gets moved into different dataframes as we merge in the future.
### We will drop our duplicates

In [38]:
rawCallData = rawCallData.drop_duplicates(['cad_event_number'], keep='first') # Drop the twinsies

In [39]:
rawCallData.drop(['general_offense_number', 'cad_cdw_id'], axis=1, inplace=True) # Drop the redundant columns

In [40]:
rawCallData.event_clearance_code = rawCallData.event_clearance_code.astype('int64', errors='raise') # Coerce
rawCallData.sort_values(by=['event_clearance_code'],inplace=True) # Sort

# So now we have a nice clean dataframe that just needs some feature engineering,
We wil start with the types of events. There are 111 different codes, let's see if we can bin these into categories based on the code. First we will make a code book to see what each code means. Then we will bin the records into general types of calls.

In [41]:
eventCode = rawCallData.event_clearance_code # Create a series from our code column
eventDesc = rawCallData.event_clearance_description # Same from our descriptions

eventBook = dict(zip(eventCode, eventDesc)) # Come together like a beatles song into a beautiful dictionary

eventBook # Look our dictionary is kinda like... a dictionary


{10: 'HOMICIDE',
 30: 'ARMED ROBBERY',
 31: 'STRONG ARM ROBBERY',
 40: 'ASSAULTS, OTHER',
 41: 'HARASSMENT, THREATS',
 42: 'HARASSMENT, THREATS - BY TELEPHONE, WRITING',
 43: 'ASSAULTS, FIREARM INVOLVED',
 49: 'ASSAULTS, GANG RELATED',
 50: 'BURGLARY - RESIDENTIAL, UNOCCUPIED',
 51: 'BURGLARY - RESIDENTIAL, OCCUPIED',
 52: 'BURGLARY - COMMERCIAL',
 53: 'BURGLARY - UNOCCUPIED STRUCTURE ON RESIDENTIAL PROPERTY',
 61: 'THEFT - AUTO ACCESSORIES',
 62: 'BICYCLE THEFT',
 63: 'THEFT - CAR PROWL',
 64: 'SHOPLIFT',
 65: 'THEFT - MISCELLANEOUS',
 71: 'AUTO THEFT',
 72: 'AUTO THEFT AND RECOVERY',
 73: 'AUTO RECOVERY',
 74: 'LICENSE PLATE THEFT OR LOSS',
 92: 'RECKLESS BURNING',
 100: 'FRAUD (INCLUDING IDENTITY THEFT)',
 101: 'FORGERY, BAD CHECKS',
 121: 'GAMBLING',
 122: 'LIQUOR VIOLATIONS (BUSINESS)',
 124: 'PORNOGRAPHY',
 125: 'PROSTITUTION',
 126: 'VICE, OTHER',
 127: 'SOAP (STAY OUT OF AREA OF PROSTITUTION) ORDER VIOLATION',
 130: 'PROPERTY DESTRUCTION',
 139: 'GANG GRAFFITI',
 141: 'LEWD CON

### The codes seem to follow a pattern of general types of crime being grouped into a ten base system. 
The system skips around a little bit and we want to hold out some of the codes as individual bins, so we can't just linspace to generate our bins. We will have to manually create a list of our bins unfortunately, then do the same for our labels. We can then add our bins and labels to the rawCallData. Then we can tackle our other categorical variable, zone_beat

In [42]:
bins = [0,10,39,49,59,69,79,92,109,129,139,149,169,171,177,179,189,199,209,229,259,269,289,299,323,339,346,
        347,359,369,379,389,449,450,495,620]
categories = ['homicide', 'robbery', 'assault', 'burglary', 'theft', 'car_theft', 'reckless_burning', 'fraud', 
          'vice', 'property', 'lewd_behavior', 'trespass', 'park_violation', 'liquor_violation', 'drive_by_shooting', 
          'narcotics', 'warrant_service', 'security_alarms', 'mental_health', 'disturbance', 'animals', 
          'suspicious_case', 'person_with_weapon', 'harbor_theft','casualty', 'harbor_boat_assist', 'harbor_dui', 
          'hazards', 'missing_person', 'missing_property', 'crowd_control', 'traffic_general', 'dui', 
          'traffic_minor', 'human_trafficking'] # Create lists to feed our cut function
rawCallData['event_type'] = pd.cut(rawCallData['event_clearance_code'], bins, labels=categories)
rawCallData['bins'] = pd.cut(rawCallData['event_clearance_code'], bins) # Add two columns with our bins and labels

In [43]:
pd.value_counts(rawCallData['event_type']) # Take a look at our totals by bin

disturbance           152722
traffic_minor         128685
suspicious_case       126012
theft                  83536
traffic_general        63670
liquor_violation       46224
security_alarms        41753
trespass               33340
burglary               26335
assault                25241
car_theft              21647
mental_health          16839
narcotics              15903
property               14435
hazards                13861
dui                    11030
missing_property       11015
fraud                  10713
warrant_service         6518
robbery                 5118
casualty                4657
park_violation          4465
missing_person          3945
lewd_behavior           2743
animals                 2394
person_with_weapon      2278
vice                    1610
harbor_boat_assist       733
drive_by_shooting        381
crowd_control            374
reckless_burning         157
homicide                  93
harbor_theft              45
harbor_dui                 5
human_traffick

### So, we have managed to cut down our 111 codes into 35 categories and binned the data as such.
We will now use the event type column to create our dummy columns and do the same with zone_beat.

In [44]:
rawCallData = pd.concat([rawCallData, pd.get_dummies(rawCallData['event_type'])], axis=1) # Convert to 16x9

In [45]:
rawCallData = pd.concat([rawCallData, pd.get_dummies(rawCallData['zone_beat'])], axis=1) # Make the zone data wide too
rawCallData.columns = rawCallData.columns.str.lower() # Fix our column names 

In [46]:
print(rawCallData.shape) # Check our work
rawCallData.head()

(878480, 133)


,cad_event_number,event_clearance_code,event_clearance_description,zone_beat,longitude,latitude,formatted_time,event_type,bins,homicide,...,u1,u2,u3,us,w,w1,w2,w3,wp,ws
1079591,17000056904,10,HOMICIDE,K3,-122.325060,47.597930,2017-02-15 21:15:45,homicide,"(0, 10]",1,...,0,0,0,0,0,0,0,0,0,0
196931,16000050525,10,HOMICIDE,D1,-122.345860,47.612990,2016-02-11 13:21:02,homicide,"(0, 10]",1,...,0,0,0,0,0,0,0,0,0,0
1253712,14000405538,10,HOMICIDE,O3,-122.319282,47.549830,2014-12-07 14:00:00,homicide,"(0, 10]",1,...,0,0,0,0,0,0,0,0,0,0
525375,17000073072,10,HOMICIDE,B3,-122.338860,47.661385,2017-02-28 17:10:27,homicide,"(0, 10]",1,...,0,0,0,0,0,0,0,0,0,0
1359327,17000298506,10,HOMICIDE,G3,-122.311530,47.595870,2017-08-13 01:56:59,homicide,"(0, 10]",1,...,0,0,0,0,0,0,0,0,0,0


# Looks like we finally have a nicely cleaned data frame, although it has some extra columns from our binning and making dummy columns.
We will go ahead and save this data frame to disk and create a codebook in markdown so anyone else who wants to work with this data in the future (most likely us) will have a nice and organized dataframe that can be used with dummy columns or without, binned events or not, and a nice record of what each column means. Then we can clean it up for our present use and change the name to just callData.

In [47]:
rawCallData.to_pickle('Data/rawCallData.pkl') # We will save it as a pickle file, since we love python so much

In [48]:
callData = rawCallData.drop(['event_clearance_code', 'event_clearance_description', 
                             'zone_beat', 'event_type', 'bins'], axis=1) # Drop 'em like they're hot 

In [49]:
del rawCallData # We have great memory, the best memory, it's yuuuge, but waste not want not
callData.head()

,cad_event_number,longitude,latitude,formatted_time,homicide,robbery,assault,burglary,theft,car_theft,...,u1,u2,u3,us,w,w1,w2,w3,wp,ws
1079591,17000056904,-122.325060,47.597930,2017-02-15 21:15:45,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
196931,16000050525,-122.345860,47.612990,2016-02-11 13:21:02,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1253712,14000405538,-122.319282,47.549830,2014-12-07 14:00:00,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
525375,17000073072,-122.338860,47.661385,2017-02-28 17:10:27,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1359327,17000298506,-122.311530,47.595870,2017-08-13 01:56:59,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Now we will tackle our other dataset, rawWeather

In [50]:
print(rawWeatherData.shape) # What have we here?
rawWeatherData.head()

(38335, 29)


,dt,dt_iso,city_id,city_name,lat,lon,temp,temp_min,temp_max,pressure,...,snow_1h,snow_3h,snow_24h,snow_today,clouds_all,weather_id,weather_main,weather_description,weather_icon,formatted_time
0,1349096400,2012-10-01 13:00:00 +0000 UTC,5809844,NaN,NaN,NaN,281.80,278.15,287.59,1027,...,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n,2012-10-01 06:00:00
1,1349186400,2012-10-02 14:00:00 +0000 UTC,5809844,NaN,NaN,NaN,281.62,278.15,286.48,1046,...,NaN,NaN,NaN,NaN,66,800,Clear,sky is clear,02d,2012-10-02 07:00:00
2,1349190000,2012-10-02 15:00:00 +0000 UTC,5809844,NaN,NaN,NaN,282.71,279.82,289.82,1026,...,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01d,2012-10-02 08:00:00
3,1349193600,2012-10-02 16:00:00 +0000 UTC,5809844,NaN,NaN,NaN,285.05,281.48,293.15,1026,...,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01d,2012-10-02 09:00:00
4,1349197200,2012-10-02 17:00:00 +0000 UTC,5809844,NaN,NaN,NaN,287.97,282.59,296.48,1027,...,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01d,2012-10-02 10:00:00


### We know for starters that we have our time column squared away already, so we can lose the old columns

In [51]:
rawWeatherData = rawWeatherData.drop(['dt', 'dt_iso'], axis=1) # Drop columns

### It appears that our temperature columns are in Kelvin,
so we will need to convert those. We only need the temp, we don't need the range for each hour, so let's drop min and max while we're at it

In [52]:
rawWeatherData = rawWeatherData.drop(['temp_min', 'temp_max'], axis=1) # Drop columns

In [53]:
rawWeatherData.temp = rawWeatherData.temp.apply(lambda x: (x*(9/5))-459.67) # Convert by multiplying 9/5 and -459.67

In [54]:
rawWeatherData.temp.head() # Check our work

0    47.570
1    47.246
2    49.208
3    53.420
4    58.676
Name: temp, dtype: float64

### That is oddly specific, let's coerce to int64

In [55]:
rawWeatherData.temp = rawWeatherData.temp.astype('int64', errors='raise') # Coerce

### We also know that we are only working with data from one city, Seattle, and we will use this data for the whole city, so we can lose the geo information as well

In [56]:
rawWeatherData = rawWeatherData.drop(['city_id', 'city_name', 'lat', 'lon', 'sea_level', 'grnd_level'], axis=1) # Drop

### No need for the weather icon designator, let's look at the ID, main, and description. 

In [57]:
print(rawWeatherData.weather_id.nunique())
print(rawWeatherData.weather_description.nunique()) # Look at how many different terms we have to work with in the data
print(rawWeatherData.weather_main.nunique())

28
30
11


### These all give us the same information, in more specific or general terms.
Looking at the code book, we can see that we have three descriptions for what the weather is doing, that goes beyond just cloud cover wind, etc. It covers conditions like fog or hail that aren't described elsewhere. Basically, we have the same scenario as when we binned the event description in our call data. Luckily the work has been done for us already, and we can drop the ID and main, then get_dummies for the description.

In [58]:
rawWeatherData = rawWeatherData.drop(['weather_main', 'weather_id', 'weather_icon'], axis =1) # Drop

In [59]:
rawWeatherData = pd.concat([rawWeatherData, pd.get_dummies(rawWeatherData['weather_description'])], axis=1) # Widen

In [60]:
rawWeatherData.columns = rawWeatherData.columns.str.lower().str.replace(" ", "_") # Clean up our new column names

### Another thing we noticed in the code book is that our wind speed is in meters per second, let's fix that

In [61]:
rawWeatherData.wind_speed = rawWeatherData.wind_speed.apply(lambda x: (x*2.2369)) # 1 Meter/sec = 2.2369 MPH

In [62]:
rawWeatherData.wind_speed # Check our work

0         0.0000
1         2.2369
2         4.4738
3         0.0000
4         4.4738
5         2.2369
6         2.2369
7         4.4738
8         4.4738
9         4.4738
10        4.4738
11        4.4738
12        2.2369
13        0.0000
14        6.7107
15        8.9476
16        2.2369
17        2.2369
18        4.4738
19        4.4738
20        0.0000
21        0.0000
22        0.0000
23        0.0000
24       13.4214
25        2.2369
26        2.2369
27        2.2369
28        2.2369
29        2.2369
          ...   
38305     4.4738
38306     4.4738
38307     8.9476
38308    11.1845
38309    11.1845
38310    11.1845
38311     4.4738
38312     4.4738
38313     8.9476
38314     8.9476
38315     8.9476
38316     8.9476
38317     8.9476
38318     8.9476
38319     8.9476
38320     4.4738
38321     4.4738
38322     2.2369
38323     2.2369
38324     6.7107
38325     6.7107
38326     4.4738
38327     4.4738
38328     4.4738
38329     4.4738
38330     4.4738
38331     4.4738
38332     6.71

### Again, oddly specific we will coerce again

In [63]:
rawWeatherData.wind_speed = rawWeatherData.wind_speed.astype('int64', axis=1) # Coerce

In [64]:
rawWeatherData.isnull().sum() # Look at what's left

temp                                0
pressure                            0
humidity                            0
wind_speed                          0
wind_deg                            0
rain_1h                         33741
rain_3h                         36685
rain_24h                        38320
rain_today                      38316
snow_1h                         38284
snow_3h                         38335
snow_24h                        38335
snow_today                      38335
clouds_all                          0
weather_description                 0
formatted_time                      0
squalls                             0
sky_is_clear                        0
broken_clouds                       0
drizzle                             0
few_clouds                          0
fog                                 0
haze                                0
heavy_intensity_drizzle             0
heavy_intensity_rain                0
heavy_intensity_shower_rain         0
heavy_snow  

### We can now see that our preciptation columns have NaN values instead of 0, so let's deal with those

In [65]:
rawWeatherData.fillna(0.00, inplace=True) # Code book says 0 values are Nan, so fillna as 0

In [66]:
rawWeatherData.isnull().sum() # Check our work

temp                            0
pressure                        0
humidity                        0
wind_speed                      0
wind_deg                        0
rain_1h                         0
rain_3h                         0
rain_24h                        0
rain_today                      0
snow_1h                         0
snow_3h                         0
snow_24h                        0
snow_today                      0
clouds_all                      0
weather_description             0
formatted_time                  0
squalls                         0
sky_is_clear                    0
broken_clouds                   0
drizzle                         0
few_clouds                      0
fog                             0
haze                            0
heavy_intensity_drizzle         0
heavy_intensity_rain            0
heavy_intensity_shower_rain     0
heavy_snow                      0
light_intensity_drizzle         0
light_intensity_shower_rain     0
light_rain    

### Now we have a nice clean dataframe with our weather data! We will save this as a pickle file as well and rename

In [67]:
rawWeatherData.to_pickle('Data/rawWeatherData.pkl')

In [68]:
weatherData = rawWeatherData

In [69]:
del rawWeatherData # Clean up again

In [70]:
weatherData.head()

,temp,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,rain_24h,rain_today,snow_1h,...,scattered_clouds,shower_rain,sky_is_clear,smoke,snow,thunderstorm,thunderstorm_with_heavy_rain,thunderstorm_with_light_rain,thunderstorm_with_rain,very_heavy_rain
0,47,1027,81,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0
1,47,1046,67,2,180,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0
2,49,1026,87,4,140,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0
3,53,1026,76,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0
4,58,1027,58,4,30,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0


# Almost there! All we have to do now is to merge our data
We have a record of the weather by the hour, and a record of every 911 call by the specific time that it occured. We want to add the pertinent weather data to each call record based on what it was in that hour of that day. We will do this by again binning our data. We will bin both dataframes by the hour of the day, and then merge on that column.

In [71]:
callData['by_hour'] = callData.formatted_time.dt.to_period('H') # This creates a column binned by the hour of the day

In [72]:
weatherData['by_hour'] = weatherData.formatted_time.dt.to_period('H') # Same for the weather data

In [73]:
df = pd.merge(callData, weatherData, on=['by_hour'], how='left') # Merge our dataframes to one

In [74]:
df.shape # Check our work

(1014521, 175)

### We mysteriously added 150k+ records to our data frame which tells us that something is clearly amiss here

In [75]:
print(weatherData.formatted_time.nunique())
print(callData.cad_event_number.nunique()) # Where did we go wrong? We must be joining on multiple records... Dups?
print(weatherData.shape)
print(callData.shape)

32443
878480
(38335, 47)
(878480, 129)


### Well, we can see here that we have the proper number of distinct records in our call data, but in our excitement to finish, we overlooked some duplicates in the weather data,
Lets go back, see where we have the overlap, make sure we aren't missing any other records, and then attempt our merge again.

In [76]:
rawWeatherData = pd.read_pickle("Data/rawWeatherData.pkl") # Good thing we saved this

In [77]:
mask = rawWeatherData.formatted_time.duplicated(keep=False) == True # Find our duplicates and display them 
rawWeatherData[mask]

,temp,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,rain_24h,rain_today,snow_1h,...,scattered_clouds,shower_rain,sky_is_clear,smoke,snow,thunderstorm,thunderstorm_with_heavy_rain,thunderstorm_with_light_rain,thunderstorm_with_rain,very_heavy_rain
197,46,1017,87,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
198,46,1017,87,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
201,45,1017,87,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
202,45,1017,87,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
205,45,1017,93,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
206,45,1017,93,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
228,48,1015,81,2,120,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
229,48,1015,81,2,120,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
230,48,1015,81,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
231,48,1015,81,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


### Here we can see that we have unique records, that overlap on the timestamp. The difference is mostly whether we consider it misty, or foggy.
We will go ahead and drop the duplicates, retaining the first records, because we prefer mist to fog.

In [78]:
rawWeatherData.drop_duplicates(subset='formatted_time', keep='first', inplace=True) # Drop the duplicate records

In [79]:
rawWeatherData.shape # Check our work

(32443, 46)

In [80]:
rawWeatherData.formatted_time.duplicated().sum() # Again, check

0

### Well, that cleared that up, but now we know we're missing some records, So let's find them and then we can either impute based on the nearest neighbors or try to get the data from our original source.

In [81]:
mask = (rawWeatherData.formatted_time.diff().dt.seconds > 3600) # Find gaps of more than 1 hour between records
rawWeatherData.formatted_time[mask] # Display

23      2012-10-03 06:00:00
25      2012-10-03 10:00:00
30      2012-10-03 16:00:00
62      2012-10-05 01:00:00
63      2012-10-05 03:00:00
85      2012-10-06 02:00:00
106     2012-10-07 01:00:00
154     2012-10-09 02:00:00
178     2012-10-10 04:00:00
189     2012-10-10 16:00:00
195     2012-10-10 23:00:00
208     2012-10-11 12:00:00
239     2012-10-12 21:00:00
472     2012-10-21 06:00:00
473     2012-10-21 15:00:00
476     2012-10-21 20:00:00
485     2012-10-22 08:00:00
560     2012-10-25 08:00:00
842     2012-11-04 03:00:00
879     2012-11-05 14:00:00
882     2012-11-05 18:00:00
897     2012-11-06 10:00:00
1029    2012-11-11 21:00:00
1132    2012-11-15 21:00:00
1175    2012-11-17 17:00:00
1395    2012-11-24 23:00:00
1438    2012-11-26 13:00:00
1517    2012-11-29 21:00:00
1530    2012-11-30 11:00:00
1597    2012-12-02 08:00:00
                ...        
28061   2016-12-05 11:00:00
28104   2016-12-06 07:00:00
28105   2016-12-06 09:00:00
28111   2016-12-06 15:00:00
28118   2016-12-06 2

### Well, that is quite the list of missing records. 
Which in hindsight, we expected to have hourly data for the past five years. 24x30x60 = 43,200 and we started with 38,000 before we removed the duplicates, so this should have been caught before. We lacked a bit of scientific skepticism about our data, but we caught our mistake in the end, now we just have to figure out how to collect these missing records, or impute them from nearest neighbors, or just go cry in the corner because cleaning the data has taken us longer than we anticipated. Well, crying would only compound our time problems and imputing is less accurate and might take longer as well. Off we go to our original data source, to track down our data.

## ....That was a bust... The data we received is all that our source can provide, so let's impute some weather data!
Since we are working with missing datetime data, pandas has some very cool tools for imputing missing data. We will first set our index to be our datetime data, as pandas expects of timeseries, then we can just resample using the option for 'H' which is hourly. Any missing hours will then be created as a new row, and the column data will be imputed from the record before it. We could alternatively perform a bfill, which would impute the data from the record after ours, or Back filling it. This is similiar to using the fillna method, although we can't select nearest as our method. Since we are most likely to have a steady state for our weather data in the short term, this will work just fine for our needs. Finally, we will reindex to make our formatted time a column again.

In [82]:
rawWeatherData.sort_values(by='formatted_time',inplace=True, ascending=True) # Put in chronological order

In [83]:
rawWeatherData.set_index('formatted_time', inplace=True) # Set time as index, which panda expects for timeseries

In [84]:
rawWeatherData = rawWeatherData.resample('H').ffill().reset_index() # Impute our missing data

In [85]:
print(rawWeatherData.isnull().sum()) # Check our work
print("Total number of duplicated times : " + str(rawWeatherData.formatted_time.duplicated().sum()))

formatted_time                  0
temp                            0
pressure                        0
humidity                        0
wind_speed                      0
wind_deg                        0
rain_1h                         0
rain_3h                         0
rain_24h                        0
rain_today                      0
snow_1h                         0
snow_3h                         0
snow_24h                        0
snow_today                      0
clouds_all                      0
weather_description             0
squalls                         0
sky_is_clear                    0
broken_clouds                   0
drizzle                         0
few_clouds                      0
fog                             0
haze                            0
heavy_intensity_drizzle         0
heavy_intensity_rain            0
heavy_intensity_shower_rain     0
heavy_snow                      0
light_intensity_drizzle         0
light_intensity_shower_rain     0
light_rain    

In [87]:
pd.set_option('display.max_columns', 200) # Set our options so we can see some more of the data by scrolling
pd.set_option('display.max_row', 50)

In [88]:
rawWeatherData.describe() # Look one more time at our data

,temp,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,rain_24h,rain_today,snow_1h,snow_3h,snow_24h,snow_today,clouds_all,squalls,sky_is_clear,broken_clouds,drizzle,few_clouds,fog,haze,heavy_intensity_drizzle,heavy_intensity_rain,heavy_intensity_shower_rain,heavy_snow,light_intensity_drizzle,light_intensity_shower_rain,light_rain,light_shower_snow,light_snow,mist,moderate_rain,overcast_clouds,proximity_thunderstorm,scattered_clouds,shower_rain,sky_is_clear,smoke,snow,thunderstorm,thunderstorm_with_heavy_rain,thunderstorm_with_light_rain,thunderstorm_with_rain,very_heavy_rain
count,44532.00000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.0,44532.0,44532.0,44532.000000,44532.0,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.0,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000,44532.000000
mean,53.54947,1018.264237,71.047090,4.376471,169.552187,0.087901,0.053837,0.000348,0.022113,0.000319,0.0,0.0,0.0,47.855969,0.0,0.068063,0.077293,0.000045,0.034380,0.012485,0.011744,0.0,0.003638,0.000067,0.000404,0.005996,0.003032,0.171427,0.000022,0.001931,0.055353,0.032606,0.134488,0.000449,0.208928,0.000090,0.168980,0.007433,0.000382,0.000135,0.000022,0.000022,0.000022,0.000561
std,12.00207,18.633710,19.752443,3.369525,101.084215,1.008810,0.562735,0.025785,2.306206,0.020639,0.0,0.0,0.0,36.812300,0.0,0.251858,0.267058,0.006702,0.182205,0.111040,0.107734,0.0,0.060205,0.008208,0.020101,0.077200,0.054976,0.376886,0.004739,0.043903,0.228671,0.177604,0.341179,0.021188,0.406548,0.009477,0.374738,0.085894,0.019535,0.011607,0.004739,0.004739,0.004739,0.023687
min,-459.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,45.00000,1015.000000,52.000000,2.000000,110.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,54.00000,1017.000000,75.000000,4.000000,190.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,40.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,61.00000,1022.000000,87.000000,6.000000,210.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,90.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,93.00000,1057.000000,100.000000,53.000000,360.000000,80.260000,24.000000,2.794000,243.332000,2.300000,0.0,0.0,0.0,100.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### It looks like we have some categorical columns with no records, we should remove them,
We can also see that we have some columns such as thunderstorm with x and rain or snow in the past x hours that don't provide enough data to be included in our dataframe, so let's drop some more columns before we merge again. We should have done that before, but we didn't really look deep enough before we hit a snag.

In [89]:
rawWeatherData.drop(['snow_today', 'squalls', 'heavy_intensity_drizzle', 'thunderstorm_with_heavy_rain', 
                     'thunderstorm_with_light_rain', 'thunderstorm_with_rain', 'light_shower_snow', 
                     'rain_1h', 'rain_3h', 'rain_24h', 'snow_1h', 'snow_3h', 'snow_24h'], 
                    axis=1, inplace=True) # Drop the others

In [92]:
rawWeatherData.sum() # Check our work

temp                           2.384665e+06
pressure                       4.534534e+07
humidity                       3.163869e+06
wind_speed                     1.948930e+05
wind_deg                       7.550498e+06
rain_today                     9.847580e+02
clouds_all                     2.131122e+06
sky_is_clear                   3.031000e+03
broken_clouds                  3.442000e+03
drizzle                        2.000000e+00
few_clouds                     1.531000e+03
fog                            5.560000e+02
haze                           5.230000e+02
heavy_intensity_rain           1.620000e+02
heavy_intensity_shower_rain    3.000000e+00
heavy_snow                     1.800000e+01
light_intensity_drizzle        2.670000e+02
light_intensity_shower_rain    1.350000e+02
light_rain                     7.634000e+03
light_snow                     8.600000e+01
mist                           2.465000e+03
moderate_rain                  1.452000e+03
overcast_clouds                5

### Okay, so now we just have to once again, save our rawWeatherData, create our by_hour column and merge!

In [93]:
rawWeatherData.to_pickle('Data/rawWeatherData.pkl') # Save to a pickle, overwriting our old file

In [94]:
weatherData = rawWeatherData # We don't really need to do this, but it's oddly satisfying

In [95]:
weatherData['by_hour'] = weatherData.formatted_time.dt.to_period('H') # Make our hourly binned column again

In [100]:
df = pd.merge(callData, weatherData, on=['by_hour'], how='inner') # Merge our weather and call data on the calldata

In [104]:
print(df.shape) # Check our work

print(df.robbery.count())

print(df.temp.count())

(878480, 162)
878480
878480


# Much Better! 

# And now all we have to do is drop our extra columns. This includes cad event number as we won't need an ID beyond the index at this point.

In [106]:
df.drop(['cad_event_number', 'by_hour', 'formatted_time_y', 'weather_description'], axis=1, inplace=True) # Forgot weather desc b4

### Rename our formatted time column to drop the _x

In [107]:
df.rename(columns={'formatted_time_x': 'formatted_time'}, inplace=True) # Rename to previous

### Let's take a look at the df to make sure it's how we want it

In [108]:
df.dtypes.unique()

array([dtype('float64'), dtype('<M8[ns]'), dtype('uint8'), dtype('int64')], dtype=object)

In [109]:
df.describe()

,longitude,latitude,homicide,robbery,assault,burglary,theft,car_theft,reckless_burning,fraud,vice,property,lewd_behavior,trespass,park_violation,liquor_violation,drive_by_shooting,narcotics,warrant_service,security_alarms,mental_health,disturbance,animals,suspicious_case,person_with_weapon,harbor_theft,casualty,harbor_boat_assist,harbor_dui,hazards,missing_person,missing_property,crowd_control,traffic_general,dui,traffic_minor,human_trafficking,99,b1,b2,b3,bs,c1,c2,c3,ccd,comm,cs,cty,d1,d2,d3,det,ds,e,e1,e2,e3,ep,es,f1,f2,f3,fs,g1,g2,g3,gs,h2,h3,inv,j1,j2,j3,js,k1,k2,k3,kcio07,ks,l1,l2,l3,lapt,ls,m1,m2,m3,ms,n,n1,n2,n3,np,ns,o1,o2,o3,os,q1,q2,q3,qs,r1,r2,r3,rs,s,s1,s2,s3,sctr1,sp,ss,tac3,trf,u1,u2,u3,us,w,w1,w2,w3,wp,ws,temp,pressure,humidity,wind_speed,wind_deg,rain_today,clouds_all,sky_is_clear,broken_clouds,drizzle,few_clouds,fog,haze,heavy_intensity_rain,heavy_intensity_shower_rain,heavy_snow,light_intensity_drizzle,light_intensity_shower_rain,light_rain,light_snow,mist,moderate_rain,overcast_clouds,proximity_thunderstorm,scattered_clouds,shower_rain,sky_is_clear,smoke,snow,thunderstorm,very_heavy_rain
count,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.00000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.00000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000,878480.000000
mean,-122.331501,47.620353,0.000106,0.005826,0.028733,0.029978,0.095092,0.024641,0.000179,0.012195,0.001833,0.016432,0.003122,0.037952,0.005083,0.052618,0.000434,0.018103,0.007420,0.047529,0.019168,0.173848,0.002725,0.143443,0.002593,0.000051,0.005301,0.000834,0.000006,0.015778,0.004491,0.012539,0.000426,0.072477,0.012556,0.146486,0.000003,0.000723,0.022429,0.020761,0.020797,0.000007,0.01712,0.015949,0.014478,0.000001,0.000009,0.000003,0.000013,0.021919,0.023250,0.021409,0.000041,0.000058,0.000076,0.025150,0.033670,0.019109,0.000014,0.000006,0.014706,0.015659,0.014242,0.000013,0.013423,0.015285,0.016042,0.000002,0.000003,0.000002,0.000008,0.014975,0.015981,0.017410,

# Hooray, now we have a nice clean, albeit _very_ wide dataframe, we can start to work now!
We will start by saving our dataframe again for future use

In [110]:
df.to_pickle('Data/weatherAndCallData.pkl') # A dataframe saved is a dataframe well earned

# Please open exploratory_data_analysis.ipynb to continue

### Click here -->[exploratory_data_analysis](exploratory_data_analysis.ipynb) <--- Click here